In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3

bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

In [15]:
# promt 1

def execute_prompt(iprovider, itype, ipharmacy):
    print("")
    print(f"[Insight for ::::: {iprovider}, {itype}, {ipharmacy} :::::]")
    print("")
    # Compare only the data based the pharmacy data for specified insurance provider and insurance type.
    prompt = '''Human:
You are an agent assistant who is recommending a patient the best Specialty Pharmacy.
Compare the pharmacy data available for specified payer and Insurance type.
The best pharmacy will have the least number of days in time to fill for each while ignoring the conversion percentage.

The patient's current Time to Fill is the lowest value of Time to Fill for 
Payer ''' + iprovider + ''', Insurance type ''' + itype + ''', 
Specialty Pharmacy ''' + ipharmacy + '''and Territory east.

Determine the patient's new options by finding the Specialty Pharmacies with the lowest value of Time to Fill for 
Payer ''' + iprovider + ''', Insurance type ''' + itype + ''' and Territory east.

Based on the above, determine if the patient's new options would provide a lower time to fill than the current time to fill.

If so, recommend the new Specialty Pharmacy to the patient.
If not, advise the patient to stay with their current Specialty pharmacy.

Assistant:
'''
    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={
            'text': prompt
        },
        retrieveAndGenerateConfiguration={
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': 'HZ2VTAKFFD',
                'inimodelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0'
            },
            'type': 'KNOWLEDGE_BASE'
        }
    )
    print(response["output"]["text"])

execute_prompt("CMS", "Medicare", "Elixir")

#execute_prompt("CMS", "Medicare", "Walgreens")

#execute_prompt("Humana", "Commercial", "Cigna")


[Insight for ::::: CMS, Medicare, Elixir :::::]

The patient's current Time to Fill is 6 days for Payer CMS, Insurance type Medicare, Specialty Pharmacy Elixir, and Territory East.

The new options with the lowest Time to Fill for Payer CMS, Insurance type Medicare, and Territory East are:
- CVS Hub: 9 days
- CVS Specialty Pharmacy: 6 days
- Senderra Rx Hub: 10 days
- Senderra Rx Specialty Pharmacy: 9 days
- Walgreens Hub: 10 days
- Walgreens Specialty Pharmacy: 8 days Among the new options, CVS Specialty Pharmacy has the same Time to Fill of 6 days as the patient's current Specialty Pharmacy Elixir.

Therefore, I would advise the patient to stay with their current Specialty Pharmacy Elixir, as switching to any of the new options would not provide a lower Time to Fill.


In [ ]:
# prompt 2

insightPrompt = '''
You are an agent assistant who is recommending a patient the best pharmacy.
Compare the pharmacy data available for specified payer and Insurance type.
The best pharmacy will have the least number of days in time to fill for each of the initiation sources while ignoring the conversion percentage.
The priority is to get prescriptions filled the fastest.
The patient's current pharmacy is Elixir.
For all the pharmacy options available for Payer CMS and Insurance type Medicare, 
can you make a recommendation for the best Specialty pharmacy for each of the initiation sources? 
'''

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        'text': insightPrompt
    },
    retrieveAndGenerateConfiguration={
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': 'HZ2VTAKFFD',
            'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0',
            'retrievalConfiguration': {
                'vectorSearchConfiguration': {
                    'overrideSearchType': 'HYBRID'
                }
            }
        },
        'type': 'KNOWLEDGE_BASE'
    }
)
print(response["output"]["text"])

In [11]:
# prompt 3

def execute_prompt(iprovider, itype, ipharmacy):
    print("")
    print(f"[Insight for ::::: {iprovider}, {itype}, {ipharmacy} :::::]")
    print("")
    # Compare only the data based the pharmacy data for specified insurance provider and insurance type.
    prompt = '''Human:
You are an agent assistant who is recommending a patient with better specialty pharmacy options based on the time to fill.
Lesser time to fill means higher recommendation.

Based on the patient's provided data for Payer ''' + iprovider + ''', Insurance type ''' + itype + ''', Specialty Pharmacy ''' + ipharmacy + '''and Territory east,
recommend a specialty pharmacy for the same insurance provider and the payer and the territory which has got a lesser TTF

Only consider the above values to generate a response. 
If you do not have sufficient information do not provide any response.

Do not create any information on your own.


Assistant:
'''
    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={
            'text': prompt
        },
        retrieveAndGenerateConfiguration={
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': 'HZ2VTAKFFD',
                'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0'
            },
            'type': 'KNOWLEDGE_BASE'
        }
    )
    print(response["output"]["text"])

execute_prompt("CMS", "Medicare", "Elixir")

#execute_prompt("CMS", "Medicare", "Walgreens")

#execute_prompt("Humana", "Commercial", "Cigna")


[Insight for ::::: CMS, Medicare, Elixir :::::]

Based on the provided data for Payer CMS, Insurance type Medicare, Specialty Pharmacy Elixir, and Territory East, the recommended specialty pharmacy for the same insurance provider, payer, and territory with a lesser Time To Fill (TTF) is CVS. According to the search results, for CMS Medicare in the East territory, CVS has a TTF of 6 for the Hub initiation source, which is lower than the TTF of 7 for Elixir Hub.


In [15]:
# prompt 4

def execute_prompt(iprovider, itype,isource, ipharmacy):
    print("")
    print(f"[Insight for ::::: {iprovider}, {itype},{isource}, {ipharmacy} :::::]")
    print("")
    # Compare only the data based the pharmacy data for specified insurance provider and insurance type.
    prompt = '''Human:
You are an agent assistant who is recommending a patient with better specialty pharmacy options based on the time to fill.
Lesser time to fill means higher recommendation.

Based on the patient's provided data for Initiation sources ''' + isource + ''',Payer ''' + iprovider + ''', Insurance type ''' + itype + ''', Specialty Pharmacy ''' + ipharmacy + '''and Territory east,
recommend a specialty pharmacy for the same insurance provider and the payer and the territory which has got a lesser TTF

Only consider the above values to generate a response.
If you do not have sufficient information do not provide any response.

Do not create any information on your own.


Assistant:
'''
    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={
            'text': prompt
        },
        retrieveAndGenerateConfiguration={
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': 'HZ2VTAKFFD',
                'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0',
          },
            'type': 'KNOWLEDGE_BASE'
        }
    )
    print(response["output"]["text"])

execute_prompt("CMS", "Medicare","Hub", "CVS")

#execute_prompt("CMS", "Medicare", "Walgreens")

#execute_prompt("Humana", "Commercial", "Cigna")


[Insight for ::::: CMS, Medicare,Hub, CVS :::::]

Based on the provided information for Initiation source Hub, Payer CMS, Insurance type Medicare, Specialty Pharmacy CVS, and Territory East, the specialty pharmacy with the lowest Time To Fill (TTF) for the same insurance provider, payer, and territory is Walgreens Hub with a TTF of 75.89.
